# NLP basic tutorial (spanish)

In [2]:
!python -m spacy download es_core_news_md

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_md-3.2.0/es_core_news_md-3.2.0-py3-none-any.whl (43.4 MB)


You should consider upgrading via the 'C:\Users\Boris Leon\anaconda3\python.exe -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [3]:
import os

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger

from wordcloud import WordCloud

import spacy
from spacy import displacy
import es_core_news_md

from pymongo import MongoClient

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
jar = "./pos-tagger/stanford-postagger/stanford-postagger-4.2.0.jar"
model = "./pos-tagger/stanford-postagger/models/spanish-ud.tagger"
os.environ["JAVAHOME"] = "/usr/bin/java"

In [5]:
nlp = es_core_news_md.load()

In [7]:
# Creating a connection to MongoDB
client = MongoClient("localhost", 27017)
db = client["news"]
collection = db["elespectador"]

In [8]:
text = []
for news in list(collection.find({}, {"title": 1, "summary": 1, "full_text": 1, "_id": 0})):
    text.append(news["title"])
    text.append(news["summary"])
    text.append(news["full_text"])

In [9]:
text = " ".join(text)

In [10]:
text

'Reforma rural, apertura de macro casos: las peticiones de víctimas en el Congreso Como cada 9 de abril, a las víctimas del conflicto armado en Colombia se les abren los micrófonos del Legislativo para que estas expongan sus preocupaciones y exijan justicia, verdad, reparación y no repetición. Este sábado, en el marco del Día Nacional de la Memoria y Solidaridad con las Víctimas, el Congreso, a la luz de la Ley 1448 de 2011, abro su recinto para que las víctimas fueran las protagonistas de la sesión plenaria, que reunió a senadores y representantes para escuchar los reclamos, preocupaciones y necesidades de las más de ocho millones de personas que han padecido el conflicto armado. El Congreso escuchó desde víctimas de grupos armados ilegales, como las exFarc y paramilitares, así como del Estado y familiares de militares desaparecidos y asesinados. El dolor y la resistencia fueron los testimonios de todas las personas que intervinieron en el pleno del Congreso, quienes aprovecharon para

In [11]:
# Word tokenization
words = word_tokenize(text)

In [12]:
words[:10]

['Reforma',
 'rural',
 ',',
 'apertura',
 'de',
 'macro',
 'casos',
 ':',
 'las',
 'peticiones']

In [13]:
# Words distribution
frec_dist = FreqDist(words)

In [14]:
# Printing the most common words
frec_dist.most_common(10)

[('de', 296),
 (',', 265),
 ('la', 171),
 ('.', 130),
 ('en', 121),
 ('y', 109),
 ('que', 104),
 ('el', 101),
 ('a', 82),
 ('del', 79)]

In [15]:
# Stop words
# Stop words are basically a set of commonly used words in any language, not just English.
# The reason why stop words are critical to many applications is that, if we remove the words that are very commonly used in a given language, we can focus on the important words instead.

stop_words = set(stopwords.words("spanish"))

In [16]:
stop_words

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

In [17]:
# Removing stop words
filtered_words = []

for w in words:
    if w.lower() not in stop_words:
        filtered_words.append(w)

In [18]:
# Words distribution for text without stopwords
frec_dist_filtered = FreqDist(filtered_words)

In [19]:
# Printing the most common words
frec_dist_filtered.most_common(10)

[(',', 265),
 ('.', 130),
 ('“', 35),
 ('”', 35),
 (':', 26),
 ('víctimas', 20),
 ('(', 19),
 (')', 18),
 ('Colombia', 15),
 (';', 12)]

In [ ]:
# Plotting a bar char for frecuencies
n = 10

plt.figure(figsize = (15, 8))
plt.barh([ w[0] for w in frec_dist_filtered.most_common(n) ], [ w[1] for w in frec_dist_filtered.most_common(n) ])
plt.gca().invert_yaxis()

In [ ]:
# Creating the wordcloud
# Word size is directly related to frecuency
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(text)

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Creating the wordcloud without stopwords
# Word size is directly related to frecuency
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(" ".join(filtered_words))

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Part of Speech (PoS)
# Model optimized for spanish: https://nlp.stanford.edu/software/
pos_tagger = StanfordPOSTagger(model, jar, encoding = "utf8")
pos_tags = pos_tagger.tag(filtered_words)

In [ ]:
pos_tags.sort(key = lambda tup: tup[1])
pos_tags

In [ ]:
len(pos_tags)

In [ ]:
pos_tags[-100:]

In [ ]:
# Named Entity Recognition (NER)
document = nlp(text)
displacy.render(document, style = "ent")